In [3]:
import numpy as np
from tqdm import tqdm
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import pandas as pd

################################################ data dimensions
_num_tuples = 1296
_num_features = 45
_label_dims = 11

################################################ load data
data = pd.read_excel('spectrum.xlsx').values
labels = pd.read_excel('temperature.xlsx').values

#data = pd.read_excel('spectrum_norm.xlsx').values
#labels = pd.read_excel('temperature_norm.xlsx').values


################################################ Define dataset function
class Dataset(object):
    def __init__(self, data):
        # m, n denote number of tuples and features respectively
        self._m = data[0].shape[0]
        self._n = data[0].shape[1]
        self._training_data = data[0]
        self._training_labels = data[1]
        
    def __len__(self):
        return self._m
    
    def __getitem__(self, idx):
        return  self._training_data[idx,:], self._training_labels[idx,:] 

    def fetch_col(self, col_index):
        return self._training_data[:, col_index]

    def shuffle(self, seed=None):
        if seed:
            np.random.seed(seed=seed)
        shuffled_indices = np.arange(self._m)
        np.random.shuffle(shuffled_indices)
        self._training_data = np.take(self._training_data, shuffled_indices, axis=0)
        self._training_labels = np.take(self._training_labels, shuffled_indices)

    @property
    def num_tuples(self):
        return self._m

    @property
    def num_features(self):
        return self._n

    @property
    def labels(self):
        return self._training_labels

    @property
    def data_table(self):
        return self._training_data
       
dataset = Dataset((data, labels))

################################################ Define model

import torch
from torch import nn

# model definition
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(45, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 11)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.leakyrelu = nn.LeakyReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        #x = self.sigmoid(x)
        return x
    def name(self):
        return 'nn'

from torch.utils.data import DataLoader
import torch.nn.functional as F

args = {'lr':1e-3, 
        'max_steps':100,
        'batch_size':50,
        'epoch':30,
        'enable_gpu':None}
train_loader = DataLoader(dataset, batch_size=args['batch_size'], shuffle=True)

import torch.optim as optim

device = torch.device("cpu")
model = NN().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=0.9)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.float().to(device), target.float().to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    logger.info('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

################################################ Run the training set 
for epoch in range(1, args['epoch'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    #test(args, model, device, test_loader)

0it [00:00, ?it/s]INFO:root:Train Epoch: 1 [0/1295 (0%)]	Loss: 394455.187500
INFO:root:Train Epoch: 1 [50/1295 (4%)]	Loss: 403246.093750
INFO:root:Train Epoch: 1 [100/1295 (8%)]	Loss: 404800.187500
INFO:root:Train Epoch: 1 [150/1295 (12%)]	Loss: 410513.562500
INFO:root:Train Epoch: 1 [200/1295 (15%)]	Loss: 397186.906250
INFO:root:Train Epoch: 1 [250/1295 (19%)]	Loss: 355207.531250
INFO:root:Train Epoch: 1 [300/1295 (23%)]	Loss: 109998.539062
INFO:root:Train Epoch: 1 [350/1295 (27%)]	Loss: 447558.031250
INFO:root:Train Epoch: 1 [400/1295 (31%)]	Loss: 403059.843750
INFO:root:Train Epoch: 1 [450/1295 (35%)]	Loss: 404767.750000
INFO:root:Train Epoch: 1 [500/1295 (38%)]	Loss: 381355.281250
INFO:root:Train Epoch: 1 [550/1295 (42%)]	Loss: 397920.812500
INFO:root:Train Epoch: 1 [600/1295 (46%)]	Loss: 398332.125000
INFO:root:Train Epoch: 1 [650/1295 (50%)]	Loss: 395813.375000
INFO:root:Train Epoch: 1 [700/1295 (54%)]	Loss: 387849.718750
INFO:root:Train Epoch: 1 [750/1295 (58%)]	Loss: 391959.062

26it [00:00, 203.06it/s]
0it [00:00, ?it/s]INFO:root:Train Epoch: 6 [0/1295 (0%)]	Loss: 2713.316650
INFO:root:Train Epoch: 6 [50/1295 (4%)]	Loss: 3394.147461
INFO:root:Train Epoch: 6 [100/1295 (8%)]	Loss: 3219.230469
INFO:root:Train Epoch: 6 [150/1295 (12%)]	Loss: 3020.749023
INFO:root:Train Epoch: 6 [200/1295 (15%)]	Loss: 3540.477295
INFO:root:Train Epoch: 6 [250/1295 (19%)]	Loss: 3663.542480
INFO:root:Train Epoch: 6 [300/1295 (23%)]	Loss: 3169.416992
INFO:root:Train Epoch: 6 [350/1295 (27%)]	Loss: 2750.956787
INFO:root:Train Epoch: 6 [400/1295 (31%)]	Loss: 4352.242676
INFO:root:Train Epoch: 6 [450/1295 (35%)]	Loss: 3272.698242
INFO:root:Train Epoch: 6 [500/1295 (38%)]	Loss: 4536.663574
INFO:root:Train Epoch: 6 [550/1295 (42%)]	Loss: 3187.384033
INFO:root:Train Epoch: 6 [600/1295 (46%)]	Loss: 2769.008545
INFO:root:Train Epoch: 6 [650/1295 (50%)]	Loss: 2777.067627
INFO:root:Train Epoch: 6 [700/1295 (54%)]	Loss: 3077.260010
INFO:root:Train Epoch: 6 [750/1295 (58%)]	Loss: 3054.937744
INF

0it [00:00, ?it/s]INFO:root:Train Epoch: 11 [0/1295 (0%)]	Loss: 3624.298096
INFO:root:Train Epoch: 11 [50/1295 (4%)]	Loss: 3468.466797
INFO:root:Train Epoch: 11 [100/1295 (8%)]	Loss: 5539.698730
INFO:root:Train Epoch: 11 [150/1295 (12%)]	Loss: 3470.432617
INFO:root:Train Epoch: 11 [200/1295 (15%)]	Loss: 3163.833252
INFO:root:Train Epoch: 11 [250/1295 (19%)]	Loss: 4010.015137
INFO:root:Train Epoch: 11 [300/1295 (23%)]	Loss: 3760.861572
INFO:root:Train Epoch: 11 [350/1295 (27%)]	Loss: 3261.418457
INFO:root:Train Epoch: 11 [400/1295 (31%)]	Loss: 3174.523438
INFO:root:Train Epoch: 11 [450/1295 (35%)]	Loss: 4433.264160
INFO:root:Train Epoch: 11 [500/1295 (38%)]	Loss: 3531.005615
INFO:root:Train Epoch: 11 [550/1295 (42%)]	Loss: 2879.980957
INFO:root:Train Epoch: 11 [600/1295 (46%)]	Loss: 3021.355713
INFO:root:Train Epoch: 11 [650/1295 (50%)]	Loss: 3366.766846
INFO:root:Train Epoch: 11 [700/1295 (54%)]	Loss: 3248.294922
INFO:root:Train Epoch: 11 [750/1295 (58%)]	Loss: 3050.931885
INFO:root:Tr

INFO:root:Train Epoch: 15 [1125/1295 (96%)]	Loss: 2782.506836
26it [00:00, 201.49it/s]
0it [00:00, ?it/s]INFO:root:Train Epoch: 16 [0/1295 (0%)]	Loss: 2934.254883
INFO:root:Train Epoch: 16 [50/1295 (4%)]	Loss: 3475.202148
INFO:root:Train Epoch: 16 [100/1295 (8%)]	Loss: 3771.920166
INFO:root:Train Epoch: 16 [150/1295 (12%)]	Loss: 3150.414062
INFO:root:Train Epoch: 16 [200/1295 (15%)]	Loss: 3161.573730
INFO:root:Train Epoch: 16 [250/1295 (19%)]	Loss: 3138.187012
INFO:root:Train Epoch: 16 [300/1295 (23%)]	Loss: 2500.130371
INFO:root:Train Epoch: 16 [350/1295 (27%)]	Loss: 3067.805420
INFO:root:Train Epoch: 16 [400/1295 (31%)]	Loss: 3032.087891
INFO:root:Train Epoch: 16 [450/1295 (35%)]	Loss: 2566.393555
INFO:root:Train Epoch: 16 [500/1295 (38%)]	Loss: 3614.268311
INFO:root:Train Epoch: 16 [550/1295 (42%)]	Loss: 3137.741455
INFO:root:Train Epoch: 16 [600/1295 (46%)]	Loss: 2933.866943
INFO:root:Train Epoch: 16 [650/1295 (50%)]	Loss: 3035.625000
INFO:root:Train Epoch: 16 [700/1295 (54%)]	Loss

INFO:root:Train Epoch: 20 [1150/1295 (88%)]	Loss: 3070.730713
INFO:root:Train Epoch: 20 [1200/1295 (92%)]	Loss: 2730.752197
INFO:root:Train Epoch: 20 [1125/1295 (96%)]	Loss: 3210.636719
26it [00:00, 201.49it/s]
0it [00:00, ?it/s]INFO:root:Train Epoch: 21 [0/1295 (0%)]	Loss: 2915.004150
INFO:root:Train Epoch: 21 [50/1295 (4%)]	Loss: 3761.441650
INFO:root:Train Epoch: 21 [100/1295 (8%)]	Loss: 2902.508545
INFO:root:Train Epoch: 21 [150/1295 (12%)]	Loss: 3099.527832
INFO:root:Train Epoch: 21 [200/1295 (15%)]	Loss: 3086.592285
INFO:root:Train Epoch: 21 [250/1295 (19%)]	Loss: 2907.179932
INFO:root:Train Epoch: 21 [300/1295 (23%)]	Loss: 3641.360840
INFO:root:Train Epoch: 21 [350/1295 (27%)]	Loss: 2656.967773
INFO:root:Train Epoch: 21 [400/1295 (31%)]	Loss: 3031.695312
INFO:root:Train Epoch: 21 [450/1295 (35%)]	Loss: 2980.959229
INFO:root:Train Epoch: 21 [500/1295 (38%)]	Loss: 3990.352051
INFO:root:Train Epoch: 21 [550/1295 (42%)]	Loss: 3206.314209
INFO:root:Train Epoch: 21 [600/1295 (46%)]	Lo

21it [00:00, 204.30it/s]INFO:root:Train Epoch: 25 [1050/1295 (81%)]	Loss: 3762.050293
INFO:root:Train Epoch: 25 [1100/1295 (85%)]	Loss: 3153.603516
INFO:root:Train Epoch: 25 [1150/1295 (88%)]	Loss: 2990.597168
INFO:root:Train Epoch: 25 [1200/1295 (92%)]	Loss: 2926.313721
INFO:root:Train Epoch: 25 [1125/1295 (96%)]	Loss: 3463.444580
26it [00:00, 204.20it/s]
0it [00:00, ?it/s]INFO:root:Train Epoch: 26 [0/1295 (0%)]	Loss: 3279.225098
INFO:root:Train Epoch: 26 [50/1295 (4%)]	Loss: 3404.394287
INFO:root:Train Epoch: 26 [100/1295 (8%)]	Loss: 4456.812012
INFO:root:Train Epoch: 26 [150/1295 (12%)]	Loss: 3232.509277
INFO:root:Train Epoch: 26 [200/1295 (15%)]	Loss: 3220.513916
INFO:root:Train Epoch: 26 [250/1295 (19%)]	Loss: 2845.111572
INFO:root:Train Epoch: 26 [300/1295 (23%)]	Loss: 4968.686035
INFO:root:Train Epoch: 26 [350/1295 (27%)]	Loss: 3074.515381
INFO:root:Train Epoch: 26 [400/1295 (31%)]	Loss: 4232.320312
INFO:root:Train Epoch: 26 [450/1295 (35%)]	Loss: 2916.556152
INFO:root:Train Epo

INFO:root:Train Epoch: 30 [950/1295 (73%)]	Loss: 3321.599365
20it [00:00, 199.94it/s]INFO:root:Train Epoch: 30 [1000/1295 (77%)]	Loss: 2909.520752
INFO:root:Train Epoch: 30 [1050/1295 (81%)]	Loss: 2833.231689
INFO:root:Train Epoch: 30 [1100/1295 (85%)]	Loss: 3552.187500
INFO:root:Train Epoch: 30 [1150/1295 (88%)]	Loss: 3066.779785
INFO:root:Train Epoch: 30 [1200/1295 (92%)]	Loss: 3048.953125
INFO:root:Train Epoch: 30 [1125/1295 (96%)]	Loss: 3913.283447
26it [00:00, 196.91it/s]
